<a href="https://colab.research.google.com/github/sanAkel/ufs_diurnal_diagnostics/blob/main/GFS/compare_model_restart_OSTIA_SST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install modules

In [ ]:
!pip install cartopy

In [ ]:
!mkdir -p data_prepared

# manually upload data from RDHPCS

!ls data_prepared/

!pwd

In [ ]:
import glob as glob
import xarray as xr
import numpy as np
import pandas as pd
from datetime import datetime

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

%matplotlib inline

## User inputs

In [ ]:
# /scratch1/NCEPDEV/climate/role.ufscpara/IC/HR5/C1152mx025/20250110/
start_date = '2019-12-02' # Assume 00 UTC

K_to_C = 273.15 # Kelvin to degC

# US North East coast
lon_s, lon_e = [-79, -74]
lat_s, lat_e = [33, 41]

cMin, cMax = [5, 25]

## Read prepared data:
- MOM6 restart that contants potential temperature.
- OSTIA SST.

In [ ]:
restart_fName = glob.glob('/content/data_prepared/ic_sst_*.nc')
print(f"Read restart file:\n{restart_fName[0]}")

print("\n")
ostia_sst_fName = glob.glob('/content/data_prepared/ostia_sst_*.nc')
print(f"Read OSTIA SST file:\n{ostia_sst_fName[0]}")

In [ ]:
# restart
ds_rst = xr.open_dataset(restart_fName[0], decode_times=False)

# fix time
ds_rst['Time'] =  pd.Timestamp(start_date)+\
pd.to_timedelta((ds_rst.Time.values-ds_rst.Time[0].values),
                unit=ds_rst.Time.attrs['units'])

In [ ]:
# ostia SST
ds_ostia = xr.open_dataset(ostia_sst_fName[0])

# ease comparison with ocean model: unit coversion
ds_ostia['analysed_sst'] = ds_ostia['analysed_sst'] - K_to_C

In [ ]:
nData = 2
plot_date = ds_rst.Time[-1].values

fig, ax_all = plt.subplots(nrows=1, ncols=nData, sharey=True, figsize=(10, 12), clear=True, subplot_kw={'projection': ccrs.PlateCarree()})

for ix, ax in enumerate(ax_all):
  ax.coastlines()
  ax.gridlines()
  ax.add_feature(cfeature.LAND, zorder=10, edgecolor='k')
  ax.set_extent([lon_s, lon_e, lat_s, lat_e])

  gl = ax.gridlines(draw_labels=True)
  gl.top_labels = False
  gl.right_labels = False
  gl.xformatter = LONGITUDE_FORMATTER
  gl.yformatter = LATITUDE_FORMATTER
  gl.xlabel_style = {'size': 10}
  gl.ylabel_style = {'size': 10}

  if ix == 0: # restart
    title_str = 'IC SST'
    im = ds_rst.sel(Time=plot_date).Temp.sel(lath=slice(lat_s, lat_e)).\
    sel(lonh=slice(lon_s, lon_e)). plot(ax=ax, transform=ccrs.PlateCarree(),
       shading='flat', vmin=cMin, vmax=cMax, cmap='Set1_r', add_colorbar=False)
  else:
    title_str = 'OSTIA SST'
    im = ds_ostia.sel(time=plot_date).analysed_sst.\
    sel(latitude=slice(lat_s, lat_e), longitude=slice(lon_s, lon_e)).\
    plot(ax=ax, transform=ccrs.PlateCarree(),
    shading='flat', vmin=cMin, vmax=cMax, cmap='Set1_r', add_colorbar=False)

  ax.text(lon_s, lat_e-1, "{}\n{}".format(title_str, plot_date),
        zorder=15, fontsize=12,color='k', transform=ccrs.PlateCarree())
  ax.set_title('')

  cbar=plt.colorbar(im, orientation='vertical', pad=0.05, shrink=0.45)
  cbar.ax.set_xlabel('[degC]')

## Compare mean monthly mean

In [ ]:
rst_mean_SST = ds_rst.Temp.sel(lath=slice(lat_s, lat_e)).\
sel(lonh=slice(lon_s, lon_e)).mean('Time')

ostia_mean_SST = ds_ostia.analysed_sst.\
sel(latitude=slice(lat_s, lat_e), longitude=slice(lon_s, lon_e)).mean('time')

In [ ]:
nData = 2

fig, ax_all = plt.subplots(nrows=1, ncols=nData, sharey=True, figsize=(10, 12), clear=True, subplot_kw={'projection': ccrs.PlateCarree()})

for ix, ax in enumerate(ax_all):
  ax.coastlines()
  ax.gridlines()
  ax.add_feature(cfeature.LAND, zorder=10, edgecolor='k')
  ax.set_extent([lon_s, lon_e, lat_s, lat_e])

  gl = ax.gridlines(draw_labels=True)
  gl.top_labels = False
  gl.right_labels = False
  gl.xformatter = LONGITUDE_FORMATTER
  gl.yformatter = LATITUDE_FORMATTER
  gl.xlabel_style = {'size': 10}
  gl.ylabel_style = {'size': 10}

  if ix == 0: # restart
    title_str = 'IC SST'
    im = rst_mean_SST.sel(lath=slice(lat_s, lat_e)).\
    sel(lonh=slice(lon_s, lon_e)). plot(ax=ax, transform=ccrs.PlateCarree(),
       shading='flat', vmin=cMin, vmax=cMax, cmap='Set1_r', add_colorbar=False)
  else:
    title_str = 'OSTIA SST'
    im = ostia_mean_SST.sel(latitude=slice(lat_s, lat_e), longitude=slice(lon_s, lon_e)).plot(ax=ax, transform=ccrs.PlateCarree(),
    shading='flat', vmin=cMin, vmax=cMax, cmap='Set1_r', add_colorbar=False)

  ax.text(lon_s, lat_e-1, "{}\n Dec 2019 mean".format(title_str),
        zorder=15, fontsize=12,color='k', transform=ccrs.PlateCarree())
  ax.set_title('')

  cbar=plt.colorbar(im, orientation='vertical', pad=0.05, shrink=0.45)
  cbar.ax.set_xlabel('[degC]')